 # IMPORT DATA

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os


In [2]:
#Creating a shortcut to access the Instacart Basket data as path

path= r'C:\\Users\\carol\\Documents\\20.06 Instacart Basket Analysis'

In [4]:
# import ords_products_merge DF 

ords_prods_merge = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','orders_products_merge.pkl'))

# QUESTION 2. FIND AGG MEAN OF 'ORDER_NUMBER' COLUMN GROUPSED BY 'DEPARTMENT_ID' FOR A SUBSET OF THE DF

In [5]:
#create subset for the 1st one million rows

df=ords_prods_merge[:1000000]

In [6]:
#check subset

df.shape

(1000000, 16)

In [7]:
# group by 'department id' agg 'order_number' mean

df.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.577493
2,17.320781
3,16.084944
4,17.530458
5,14.763075
6,16.658449
7,17.031590
8,15.076662


# QUESTION 4. CREATING A LOYALTY FLAG ( USING TRANSFORM() AND LOC() FUNCTIONS)

In [8]:
# Creating new column \"max_order\"

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\carol\AppData\Local\Temp\ipykernel_12276\425958132.py:3: FutureWarning: The provided callable <function max at 0x000001571C0D5BC0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [9]:
#Creating Loyalty flag

ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [10]:
# counting the results

ords_prods_merge['loyalty_flag'].value_counts(dropna=False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

# QUESTION 5. CHECK

In [11]:
# Performing statistics on prices related to Loyalty flag

ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

# QUESTION 6. CREATE A SPENDING FLAG FOR EACH USER BASED ON CRITERIA

In [12]:
# Creating  new column 'mean_product_price'

ords_prods_merge['mean_product_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\carol\AppData\Local\Temp\ipykernel_12276\3189927059.py:3: FutureWarning: The provided callable <function mean at 0x000001571C0D65C0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['mean_product_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [13]:
#Checking data

ords_prods_merge[['user_id', 'mean_product_price']].head(10)

,user_id,mean_product_price
0,138,6.935811
1,138,6.935811
2,709,7.930208
3,764,4.972414
4,764,4.972414
5,777,6.935398
6,825,5.957576
7,910,6.680000
8,1052,7.162500
9,1052,7.162500


In [14]:
# Creating Spending flag for high and low spender using .loc()

ords_prods_merge.loc[ords_prods_merge['mean_product_price'] >= 10, 'spending_flag'] = 'High spender'
ords_prods_merge.loc[ords_prods_merge['mean_product_price'] < 10, 'spending_flag'] = 'Low spender'

In [15]:
# Checking data

ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

In [16]:
# Checking newly created spending flag 

ords_prods_merge[['user_id', 'mean_product_price', 'spending_flag']].head(5)

,user_id,mean_product_price,spending_flag
0,138,6.935811,Low spender
1,138,6.935811,Low spender
2,709,7.930208,Low spender
3,764,4.972414,Low spender
4,764,4.972414,Low spender


# QUESTION 7. CREATE AN ORDER FREQUECY FLAG ACCORDING TO THE MEDIAN 'DAYS_SINCE_PRIOR_ORDER'

In [17]:
#Creating new column 'median_days_between_orders'

ords_prods_merge['median_days_between_orders'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\carol\AppData\Local\Temp\ipykernel_12276\433868533.py:3: FutureWarning: The provided callable <function median at 0x000001571C2B6480> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days_between_orders'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [18]:
# Check newly created flag 

ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders']].head(10)

,user_id,days_since_prior_order,median_days_between_orders
0,138,3.0,8.0
1,138,20.0,8.0
2,709,6.0,8.0
3,764,NaN,9.0
4,764,9.0,9.0
5,777,26.0,11.0
6,825,30.0,20.0
7,910,30.0,6.0
8,1052,19.0,10.0
9,1052,15.0,10.0


In [21]:
# identifying median value on days_since_prior_order 
median_value = ords_prods_merge['days_since_prior_order'].median()


In [20]:
#check calculated median value

median_value

8.0

In [22]:
# replace median value to fill Nan on column days_since_prior_order 

ords_prods_merge['days_since_prior_order'].fillna(median_value, inplace=True)

C:\Users\carol\AppData\Local\Temp\ipykernel_12276\2560985447.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ords_prods_merge['days_since_prior_order'].fillna(median_value, inplace=True)


In [23]:
# Check changes (Nan) newly created flag 

ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders']].head(10)

,user_id,days_since_prior_order,median_days_between_orders
0,138,3.0,8.0
1,138,20.0,8.0
2,709,6.0,8.0
3,764,8.0,9.0
4,764,9.0,9.0
5,777,26.0,11.0
6,825,30.0,20.0
7,910,30.0,6.0
8,1052,19.0,10.0
9,1052,15.0,10.0


In [24]:
# Creating frequency flag based on median_days_between:orders .loc()

ords_prods_merge.loc[ords_prods_merge['median_days_between_orders'] > 20, 'order_frequency'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_between_orders'] <= 20) & (ords_prods_merge['median_days_between_orders'] > 10), 'order_frequency'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_between_orders'] <= 10, 'order_frequency'] = 'Frequent customer'

In [25]:
# Checking data

ords_prods_merge['order_frequency'].value_counts(dropna = False)

order_frequency
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

In [26]:
ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders', 'order_frequency']].head(10)

,user_id,days_since_prior_order,median_days_between_orders,order_frequency
0,138,3.0,8.0,Frequent customer
1,138,20.0,8.0,Frequent customer
2,709,6.0,8.0,Frequent customer
3,764,8.0,9.0,Frequent customer
4,764,9.0,9.0,Frequent customer
5,777,26.0,11.0,Regular customer
6,825,30.0,20.0,Regular customer
7,910,30.0,6.0,Frequent customer
8,1052,19.0,10.0,Frequent customer
9,1052,15.0,10.0,Frequent customer


# EXPORTING DATA

In [27]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_grouped2.pkl'))